# Scratch Paper:

In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE, ADASYN
import numpy as np
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)



/Users/Thomas/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
dfi = pd.read_csv('aac_intakes_outcomes.csv.zip')
display(dfi.head())
dfi.info()

,age_upon_outcome,animal_id_outcome,date_of_birth,outcome_subtype,outcome_type,sex_upon_outcome,age_upon_outcome_(days),age_upon_outcome_(years),age_upon_outcome_age_group,outcome_datetime,outcome_month,outcome_year,outcome_monthyear,outcome_weekday,outcome_hour,outcome_number,dob_year,dob_month,dob_monthyear,age_upon_intake,animal_id_intake,animal_type,breed,color,found_location,intake_condition,intake_type,sex_upon_intake,count,age_upon_intake_(days),age_upon_intake_(years),age_upon_intake_age_group,intake_datetime,intake_month,intake_year,intake_monthyear,intake_weekday,intake_hour,intake_number,time_in_shelter,time_in_shelter_days
0,10 years,A006100,2007-07-09 00:00:00,NaN,Return to Owner,Neutered Male,3650,10.0,"(7.5, 10.0]",2017-12-07 14:07:00,12,2017,2017-12,Thursday,0,1.0,2007,7,2017-12,10 years,A006100,Dog,Spinone Italiano Mix,Yellow/White,Colony Creek And Hunters Trace in Austin (TX),Normal,Stray,Neutered Male,1,3650,10.0,"(7.5, 10.0]",2017-12-07 00:00:00,12,2017,2017-12,Thursday,14,1.0,0 days 14:07:00.000000000,0.588194
1,7 years,A006100,2007-07-09 00:00:00,NaN,Return to Owner,Neutered Male,2555,7.0,"(5.0, 7.5]",2014-12-20 16:35:00,12,2014,2014-12,Saturday,16,2.0,2007,7,2014-12,7 years,A006100,Dog,Spinone Italiano Mix,Yellow/White,8700 Research Blvd in Austin (TX),Normal,Public Assist,Neutered Male,1,2555,7.0,"(5.0, 7.5]",2014-12-19 10:21:00,12,2014,2014-12,Friday,10,2.0,1 days 06:14:00.000000000,1.259722
2,6 years,A006100,2007-07-09 00:00:00,NaN,Return to Owner,Neutered Male,2190,6.0,"(5.0, 7.5]",2014-03-08 17:10:00,3,2014,2014-03,Saturday,17,3.0,2007,7,2014-03,6 years,A006100,Dog,Spinone Italiano Mix,Yellow/White,8700 Research in Austin (TX),Normal,Public Assist,Neutered Male,1,2190,6.0,"(5.0, 7.5]",2014-03-07 14:26:00,3,2014,2014-03,Friday,14,3.0,1 days 02:44:00.000000000,1.113889
3,10 years,A047759,2004-04-02 00:00:00,Partner,Transfer,Neutered Male,3650,10.0,"(7.5, 10.0]",2014-04-07 15:12:00,4,2014,2014-04,Monday,15,1.0,2004,4,2014-04,10 years,A047759,Dog,Dachshund,Tricolor,Austin (TX),Normal,Owner Surrender,Neutered Male,1,3650,10.0,"(7.5, 10.0]",2014-04-02 15:55:00,4,2014,2014-04,Wednesday,15,1.0,4 days 23:17:00.000000000,4.970139
4,16 years,A134067,1997-10-16 00:00:00,NaN,Return to Owner,Neutered Male,5840,16.0,"(15.0, 17.5]",2013-11-16 11:54:00,11,2013,2013-11,Saturday,11,1.0,1997,10,2013-11,16 years,A134067,Dog,Shetland Sheepdog,Brown/White,12034 Research Blvd in Austin (TX),Injured,Public Assist,Neutered Male,1,5840,16.0,"(15.0, 17.5]",2013-11-16 09:02:00,11,2013,2013-11,Saturday,9,1.0,0 days 02:52:00.000000000,0.119444


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79672 entries, 0 to 79671
Data columns (total 41 columns):
age_upon_outcome              79672 non-null object
animal_id_outcome             79672 non-null object
date_of_birth                 79672 non-null object
outcome_subtype               36348 non-null object
outcome_type                  79662 non-null object
sex_upon_outcome              79671 non-null object
age_upon_outcome_(days)       79672 non-null int64
age_upon_outcome_(years)      79672 non-null float64
age_upon_outcome_age_group    79672 non-null object
outcome_datetime              79672 non-null object
outcome_month                 79672 non-null int64
outcome_year                  79672 non-null int64
outcome_monthyear             79672 non-null object
outcome_weekday               79672 non-null object
outcome_hour                  79672 non-null int64
outcome_number                79672 non-null float64
dob_year                      79672 non-null int64
dob_month 

In [3]:
dfi['count'].value_counts()

1    79672
Name: count, dtype: int64

In [4]:
dfii = pd.read_csv('aac_intakes.csv.zip')

In [5]:
dfx = dfii[['animal_id', 'name', 'breed']]

In [6]:
df = dfi.set_index('animal_id_intake').join(dfx.set_index('animal_id'), 
                                            rsuffix = '_in', how = 'left')
#df[['name_in', 'animal_id_intake', 'animal_id_in']].tail()
#df[['name', 'breed', 'breed_in']].tail(20)

df['unique_id'] = df['animal_id_outcome'] + df['intake_number'].astype('str')

In [7]:
df = df.drop_duplicates(subset = ['unique_id'])

In [8]:
df = df[df['outcome_type'].isin(['Adoption', 'Transfer', 'Return to Owner'])]
df['outcome_type'].value_counts()

Adoption           33583
Transfer           23791
Return to Owner    14784
Name: outcome_type, dtype: int64

In [9]:
df.name.fillna(0, inplace = True)
df['has_name'] = df['name'].map(lambda x: 1 if not x == 0 else 0)

In [10]:
df.animal_type.value_counts()

Dog      43538
Cat      27568
Other      833
Bird       219
Name: animal_type, dtype: int64

In [11]:
df = df[df.animal_type != 'Bird']
df = df[df.animal_type != 'Other']
df.animal_type.value_counts()

Dog    43538
Cat    27568
Name: animal_type, dtype: int64

In [12]:
df.sex_upon_outcome.dropna(inplace = True)
df.sex_upon_intake.dropna(inplace = True)

col_to_keep = ['sex_upon_intake', 'age_upon_intake_(years)', 
               'time_in_shelter_days','animal_type', 'breed',
               'color', 'found_location', 'intake_condition', 
               'intake_type', 'intake_number', 'has_name',
               'outcome_datetime', 'outcome_type']
df_eda = df[col_to_keep]
df_eda.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71106 entries, A006100 to A769066
Data columns (total 13 columns):
sex_upon_intake            71105 non-null object
age_upon_intake_(years)    71106 non-null float64
time_in_shelter_days       71106 non-null float64
animal_type                71106 non-null object
breed                      71106 non-null object
color                      71106 non-null object
found_location             71106 non-null object
intake_condition           71106 non-null object
intake_type                71106 non-null object
intake_number              71106 non-null float64
has_name                   71106 non-null int64
outcome_datetime           71106 non-null object
outcome_type               71106 non-null object
dtypes: float64(3), int64(1), object(9)
memory usage: 7.6+ MB


In [13]:
df_eda['sex_upon_intake'].fillna('Unknown', inplace = True)

df_eda.isna().sum()

sex_upon_intake            0
age_upon_intake_(years)    0
time_in_shelter_days       0
animal_type                0
breed                      0
color                      0
found_location             0
intake_condition           0
intake_type                0
intake_number              0
has_name                   0
outcome_datetime           0
outcome_type               0
dtype: int64

In [14]:
dfi = df_eda

# Dummy Variables:

In [15]:
# sex_upon_intake
# Dummy variable, no change
# Maybe split into male/female and intact or no

In [16]:
dfi.sex_upon_intake.value_counts()

Intact Male      23618
Intact Female    22387
Neutered Male    11975
Spayed Female    10648
Unknown           2478
Name: sex_upon_intake, dtype: int64

In [17]:
dfi['sex'] = np.where(dfi['sex_upon_intake'].str.contains('Male'), 1, 0)
dfi['intact'] = np.where(dfi['sex_upon_intake'].str.contains('Intact'), 1, 0)
dfi['unk_sex'] = np.where(dfi['sex_upon_intake'].str.contains('Unknown'), 1, 0)
# dfi.drop(columns = 'sex_upon_intake', axis = 1, inplace = True)

In [18]:
# age_upon_intake_(years)
# scale

In [19]:
# time_in_shelter_days 
# Scale

In [20]:
# animal_type 
# Dummy for dog or cat
dfi['dog'] = dfi['animal_type'].map(lambda x: 1 if x == 'Dog' else 0)
dfi['cat'] = dfi['animal_type'].map(lambda x: 1 if x == 'Cat' else 0)

In [21]:
# breed
# Dummy for threshold of .05
def dum_sign(dummy_col, threshold=0.1):
    # removes the bind
    dummy_col = dummy_col.copy()
    # Ratio of dummy to rest of column
    count = pd.value_counts(dummy_col) / len(dummy_col)
    # Condition
    mask = dummy_col.isin(count[count > threshold].index)
    # Replaces 
    dummy_col[~mask] = "others"

    return pd.get_dummies(dummy_col, prefix=dummy_col.name)

# df_breed = dum_sign(dfi['breed'], threshold = .01)
df_breed = dum_sign(dfi['breed'], threshold = .005)
dfi = pd.concat([dfi, df_breed], axis = 1)

In [22]:
# color
# Dummy for top 10-20
# df_col = dum_sign(dfi['color'], threshold = .025)
df_col = dum_sign(dfi['color'], threshold = .01)
dfi = pd.concat([dfi, df_col], axis = 1)

In [23]:
# found_location
# Dummy for top ten, or possibly just Austin or outside Austin
# df_loc = dum_sign(dfi['found_location'], threshold = .01)
df_loc = dum_sign(dfi['found_location'], threshold = .005)
dfi = pd.concat([dfi, df_loc], axis = 1)

In [24]:
# intake_condition
# Regular dummy variables
cond_dummy = pd.get_dummies(dfi['intake_condition'])
dfi = pd.concat([dfi, cond_dummy], axis = 1)

In [25]:
# intake_type         
# Regular dummy variables
int_type_dummy = pd.get_dummies(dfi['intake_type'])
dfi = pd.concat([dfi, int_type_dummy], axis = 1)

In [26]:
# intake_number
# Dummy for 1, 2, 3, and 4+
dfi['count_1'] = dfi['intake_number'].map(lambda x: 1 if x == 1 else 0)
dfi['count_2'] = dfi['intake_number'].map(lambda x: 1 if x == 2 else 0)
dfi['count_3'] = dfi['intake_number'].map(lambda x: 1 if x == 3 else 0)
dfi['count_4_plus'] = dfi['intake_number'].map(lambda x: 1 if x > 3 else 0)

In [27]:
# has_name               
# Good to go

In [28]:
# outcome_datetime      
# TBD

In [29]:
# outcome_type  
# Need to convert to numericals in the same column. . . 
# Adoption = 1
# Transfer = 2
# Return to owner = 3
dfi.outcome_type.unique()
#dfi['outcome'] = dfi['outcome_type'].map(lambda x: 1 if x == 'Return to Owner' elif x == '')
#dfi['outcome'] = dfi['outcome_type'].map(lambda x: 2 if x == 'Transfer')
#dfi['outcome'] = dfi['outcome_type'].map(lambda x: 3 if x == 'Adoption')
dfi['outcome'] = dfi['outcome_type'].map(lambda x: 1 if x == 'Return to Owner' 
                                         else (2 if x == 'Transfer' 
                                               else 3))

# Testing Various ML Algos:

## X, y:

In [30]:
# DROPPING COLUMNS TO SCALE
'''
X = dfi.drop(['outcome', 'outcome_type', 'sex_upon_intake', 'age_upon_intake_(years)',
              'time_in_shelter_days', 'animal_type', 'breed', 
              'color', 'found_location', 'intake_condition', 'intake_type', 
              'intake_number', 'outcome_datetime'], axis = 1)
y = dfi['outcome']
'''

"\nX = dfi.drop(['outcome', 'outcome_type', 'sex_upon_intake', 'age_upon_intake_(years)',\n              'time_in_shelter_days', 'animal_type', 'breed', \n              'color', 'found_location', 'intake_condition', 'intake_type', \n              'intake_number', 'outcome_datetime'], axis = 1)\ny = dfi['outcome']\n"

In [31]:
# NOT DROPPING COLUMNS TO SCALE 
# 'age_upon_intake_(years)'
# 'time_in_shelter_days',
# '''
X = dfi.drop(['outcome', 'outcome_type', 'sex_upon_intake',
               'animal_type', 'breed', 'color', 
              'found_location', 'intake_condition', 'intake_type', 
              'intake_number', 'outcome_datetime'], axis = 1)
y = dfi['outcome']
# '''

## Train Test Split:

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                    random_state=123)

In [33]:
# SMOTE
X_train_resampled, y_train_resampled = SMOTE().fit_sample(X_train, y_train) 

# Preview resampled sample class distribution
print('\n')
print(pd.Series(y_train_resampled).value_counts()) 



3    26569
2    26569
1    26569
dtype: int64


## Bagged Trees:

In [34]:
'''
bagged_tree =  BaggingClassifier(DecisionTreeClassifier(criterion='gini', 
                                                        max_depth=10), 
                                                        n_estimators=200)
bagged_tree.fit(X_train, y_train)
'''

"\nbagged_tree =  BaggingClassifier(DecisionTreeClassifier(criterion='gini', \n                                                        max_depth=10), \n                                                        n_estimators=200)\nbagged_tree.fit(X_train, y_train)\n"

### BT Accuracy:

In [35]:
# bagged_tree.score(X_test, y_test)

## KNN:

In [36]:
# clf = KNeighborsClassifier(n_neighbors=10)

# Fit the classifier
# clf.fit(X_train, y_train)

### KNN Accuracy:

In [37]:
#y_pred = clf.predict(X_test)

In [38]:
#clf.score(X_test, y_test)

## Random Forest:

### Random Forest Pipeline Test:

Testing this with intake age and time in shelter.  If it crashes and burns, add those columns back to the drop list for X.

In [39]:
scaled_pipeline_1 = Pipeline([('ss', StandardScaler()), 
                              ('forest', RandomForestClassifier(
                                  n_estimators=150, 
                                  max_depth= 19, 
                                  class_weight='balanced', 
                                  random_state = 123))])

scaled_pipeline_1.fit(X_train, y_train)
scaled_pipeline_1.score(X_test, y_test)

0.793348333567712

In [40]:
pred2 = scaled_pipeline_1.predict(X_test)
print(confusion_matrix(y_test, pred2))

[[2446  141  351]
 [ 148 2808 1652]
 [ 173  474 6029]]


## SMOTE TEST:

In [55]:
smote_rf = Pipeline([('ss', StandardScaler()), 
                     ('forest', RandomForestClassifier(n_estimators=200, 
                                                       max_depth= 23, 
                                                       random_state = 123))])

smote_rf.fit(X_train_resampled, y_train_resampled)
smote_rf.score(X_test, y_test)

0.7947546055407115

In [56]:
pred3 = smote_rf.predict(X_test)
print(confusion_matrix(y_test, pred3))

[[2416  145  377]
 [ 115 2802 1691]
 [ 131  460 6085]]


In [43]:
# 0.7920826887920124
# [[2426  144  368]
# [ 119 2737 1752]
# [ 147  427 6102]]

In [57]:
smote_rf

Pipeline(memory=None,
         steps=[('ss',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('forest',
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=23,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=200, n_jobs=None,
                                        oob_score=False, random_state=123,
                                        verbose=0, warm_start=False))],
         verbose=False)

## XGBoost:

In [44]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [45]:
'''
xg = xgb.XGBClassifier(learning_rate = .05, max_depth = 10, n_estimators = 200)
xg.fit(X_train, y_train)
training_preds = xg.predict(X_train)
val_preds = xg.predict(X_test)
training_accuracy = accuracy_score(y_train, training_preds)
val_accuracy = accuracy_score(y_test, val_preds)

print("Training Accuracy: {:.4}%".format(training_accuracy * 100))
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))
'''

'\nxg = xgb.XGBClassifier(learning_rate = .05, max_depth = 10, n_estimators = 200)\nxg.fit(X_train, y_train)\ntraining_preds = xg.predict(X_train)\nval_preds = xg.predict(X_test)\ntraining_accuracy = accuracy_score(y_train, training_preds)\nval_accuracy = accuracy_score(y_test, val_preds)\n\nprint("Training Accuracy: {:.4}%".format(training_accuracy * 100))\nprint("Validation accuracy: {:.4}%".format(val_accuracy * 100))\n'

In [46]:
'''
pred = xg.predict(X_test)
print(confusion_matrix(y_test, pred))
'''

'\npred = xg.predict(X_test)\nprint(confusion_matrix(y_test, pred))\n'